# Inledning
## Importera Moduler
Först importerar vi lämpliga moduler som behövs i analysen. Det är numpy, scipy och pandas för grundläggande datahantering och statistik; matplotlib och seaborn för plottning samt statsmodels för regressionen.

In [ ]:
#För att installera moduleran först avkommentera följande rad
#%pip install numpy scipy pandas matplotlib seaborn statsmodels

import numpy as np
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

## Läs in data
Ladda ner datafilen från canvas sidan och lägga csv-filen i samma katalog som ditt Python script, eller ange en lämplig relativ sökväg. Se laborationern för hur vi hanterade data. 

Vi skriver också ut data för att få en översikt av variablerna.

In [ ]:

fisk = pd.read_csv('fisk.csv', encoding='utf-8')
print( fisk )

Vi kan också gör en describe på data materialet (include='all' ser till att även Art so är en text variabel kommer med).

In [ ]:
print( fisk.describe(include='all') )

För Art variabeln kan det vara intressant med antalet olika arter.

In [ ]:
print(fisk.Art.value_counts())

## Plotta data
Vi kan nu illustrera data. Vi börjar med **Vikt** som funktion av **Langd** och färglägger med Art.

In [ ]:
sns.scatterplot(fisk, x='Langd', y='Vikt', hue='Art')

Vi kan också plotta med logaritmisk y-axel (eller x-axel)

In [ ]:
sns.scatterplot(fisk, x='Langd', y='Vikt', hue='Art')
plt.yscale('log')

## Dela data in träning och test
För att dela data i träning och utvärdering så använder vi **sample** funktionen i pandas för att slumpmässigt välja 80% av observationerna. Sen använder vi **drop** för att konstruera ett utvärderingsset som **inte** innehåller träningsdata. Notera att **sample** har en extra parameter **random_state** som kan användas för att bestämma vilka slumptal som ska användas så att tränningsdata alltid blir samma när man kör koden.

In [ ]:
fisk_train = fisk.sample(frac=0.8, random_state=1) ##Använd ert grupp-nummer som random_state
fisk_test = fisk.drop(fisk_train.index)
print( fisk_train.describe(include='all') )
print( fisk_test.describe(include='all') )

## Inledande enkel regression
Det finns minst två sätt att göra regression i python funktioner från scikit-learn som använder matris formerna direkt och kräver att användaren konstruera X matrisen för hand
**sklearn.linear_model.LinearRegression()**. Ett bättre alternativ är statsmodels regressions funktioner som direkt omvandlar formler till lämpliga X och Y matriser.
**statsmodels.formula.api.ols()**. 

Vi kommer nu använda **statsmodels** för att demonstrerar en enkel regression där **Vikt** enbart förklaras av **Langd**.

In [ ]:
help( smf.ols )

In [ ]:

res = smf.ols(formula='Vikt ~ Langd', data=fisk_train)
print( res.fit().summary() )

Notera att **smf.ols** själv lägger till ett intercept så *Vikt ~ Langd* ger modellen 

$Vikt_i = \beta_0 + \beta_1 Langd_i + \epsilon_i$

För att anpassa en modell utan $\beta_0$ kan vi använda *Vikt ~ Langd+0* eller *Vikt ~ Langd-1*.

Resultaten innehåller skattade parametrar och deras osäkerheter. Andra halvan är en utvärdering av om residualerna är normalfördelade. För normalfördelade residualer bör vi ha
* Prob(Omnibus) > 0.05
* Skew=0
* Kurtosis=3
* Prob(JB) > 0.05 

Här är Python anningen pettig; regressionen fungerar rimligt bra även för mindre avvikelser från normalfördelade residualer. Vill vi trasformera y eller x värden kan transformen tas med i formlen, tänk på att de flesta relevanta funktioner finns i numpy. Exempel
* *formula = np.log(Vikt) ~ Langd* för log-transformation av vikten
* *formula = Vikt ~ Langd + I(Langd**2)* för ett andragrads polynom av längden

### Regresion resultat
Vi vill nu analysera resultatet av regressionen. De anpassade variablerna finns i *res.fit().params*, och kan användas för att illustrerar anpassningen.

In [ ]:
beta = res.fit().params
sns.scatterplot(fisk_train, x='Langd', y='Vikt', hue='Art')
#hämta under gräns för x initial punkt för linjen.
x_min = plt.gca().get_xlim()[0]
plt.axline((x_min,beta.Intercept+x_min*beta.Langd),slope=beta.Langd) #start punkt (x_min,intercept+x_min*beta_1) och lutning slope

In [ ]:
fisk_train['Langd3'] = fisk_train['Langd']**3

res = smf.ols(formula='Vikt ~ Langd3', data=fisk_train).fit()
print(res.summary())


Vi kan nu plocka ut lite olika värden.

In [ ]:
#skattade parameterar
res.fit().params
#konfidens intervall för parametrarna
res.fit().conf_int()
#Kovariansmatris för parametrarna
res.fit().cov_params()
#anpassade värden
yhat = res.fit().fittedvalues
#residualer
epsilon = res.fit().resid

Ett alternativ är att studera hur residualerna och anpassade värden relaterar till resten av data.

In [ ]:
fisk_train['residualer'] = res.fit().resid
fisk_train['yhat'] = res.fit().fittedvalues

sns.scatterplot(fisk_train, x='yhat', y='Vikt', hue='Art')
#hämta under gräns för x och y och använd största av dessa som initial punkt för linjen.
xy_min = np.max((plt.gca().get_xlim()[0],plt.gca().get_ylim()[0]))
plt.axline((xy_min,xy_min),slope=1,color='red') #y=x referens linje

In [ ]:
sns.histplot(fisk_train, x='residualer', stat='density', kde=True)

In [ ]:
qqplot = stats.probplot(fisk_train.residualer, dist="norm", fit=True, plot=plt)

In [ ]:
sns.residplot(fisk_train, x='yhat', y='Vikt', lowess=True)

In [ ]:
sns.pairplot(fisk_train, y_vars='residualer')

För att jämföra med utvärderings data så använder vi först modellen för att prediktera vikterna och addera det som en kolumn i *fisk_test*

In [ ]:
fisk_test['yhat'] = res.fit().predict(fisk_test)
#scatter plot
sns.scatterplot(fisk_test, x='yhat', y='Vikt', hue='Art')
xy_min = np.max((plt.gca().get_xlim()[0],plt.gca().get_ylim()[0]))
plt.axline((xy_min,xy_min),slope=1,color='red')
#MSE
print( np.mean( (fisk_test.Vikt - fisk_test.yhat)**2 ) )
#samma plotar som ovan kan användas även för utvärderingsdatan.

### Transformerade y-variabler
Om Y variabeln transformeras kan vi inte längre bra rita en rak linje utan behöver prediktera värden som vi kan addera till ploten. Först skappar vi en data frame med prediktions värden

In [ ]:
Pred_G = pd.DataFrame({'Art': fisk_train.Art.unique()[0],
                       'Langd': np.arange(min(fisk_train.Langd),max(fisk_train.Langd)+1)})
Pred_R = pd.DataFrame({'Art': fisk_train.Art.unique()[1],
                       'Langd': np.arange(min(fisk_train.Langd),max(fisk_train.Langd)+1)})
#gemensam data.frame för alla prediktioner
Pred = pd.concat( [Pred_G,Pred_R] )
#titta på data
print(Pred)


Vi kan nu göra en regression och prediktera för alla värden i **Pred** och använda dessa för att rita in anpassningen.

In [ ]:
#regression
res_logV = smf.ols(formula='np.log(Vikt) ~ Langd', data=fisk_train)
print( res_logV.fit().summary() )
#prediktion, kom ihåg att transformera tillbaka
Pred['yhat'] = np.exp( res_logV.fit().predict(Pred) )

sns.scatterplot(fisk_train, x='Langd', y='Vikt', hue='Art')
sns.lineplot(Pred, x='Langd', y='yhat', hue='Art')

Alternativt kan vi plotta med logaritmisk y-axel

In [ ]:
#prediktion, i log-skala
sns.scatterplot(fisk_train, x='Langd', y='Vikt', hue='Art')
sns.lineplot(Pred, x='Langd', y='yhat', hue='Art')
plt.yscale('log')

För residual analysen måste vi komma ihåg att det är residualerna från

$\log(Vikt_i) = \beta_0 + \beta_1 Langd_i + \epsilon_i$

som ska vara normalfördelade.

In [ ]:
#regression residualer och anpassade värden (log(Vikt)-skala) från res
fisk_train['residualer_logV'] = res_logV.fit().resid
fisk_train['yhat_logV'] = res_logV.fit().fittedvalues

##subplots
fig, axs = plt.subplots(1, 2)
#prediktioner mot värden
sns.scatterplot(fisk_train, x='yhat_logV', y=np.log(fisk_train.Vikt), hue='Art', ax=axs[0])
xy_min = min( (axs[0].get_xlim()[0], axs[0].get_ylim()[0]) )
axs[0].axline((xy_min,xy_min),slope=1)
##qqplot
qqplot = stats.probplot(fisk_train.residualer_logV, dist="norm", fit=True, plot=axs[1])

#eller för hand
epsilon = np.log(fisk_train.Vikt) - res_logV.fit().fittedvalues
#vilket ger samma resultat
print(epsilon-fisk_train['residualer_logV'])

# Tips för projektet
1. Gör ett rimligt modelval och skatta modellen, anpassa koden ovan för model utvärdering. Mer komplicerade formler ger olika lutning eller intercept mellan arterna. Alternativ att fundera på är:
    * *Y ~ A + B*     ger modellen $Y_i = \beta_0 + \beta_1 \cdot A_i + \beta_2 \cdot B_i + \epsilon_i$
    * *Y ~ A\*B*       ger modellen $Y_i = \beta_0 + \beta_1 \cdot A_i + \beta_2 \cdot B_i + \beta_3 \cdot (A_i \cdot B_i) + \epsilon_i$
    * *Y ~ A:B*       ger modellen $Y_i = \beta_0 + \beta_1 \cdot (A_i \cdot B_i) + \epsilon_i$
2. Undersök de skattade parametrarna och tolka dessa
3. För prediktion så behöver ni först konstruera en *DataFrame* som innehåller data som ni vill prediktera för:

In [ ]:
U_0 = pd.DataFrame({'Art' : [?, ?],
                    'Langd' : [?, ?]})

4. Givet en ny DataFrame kan prediktions funktionerna användas för att få punkt-prediktioner, varianser och intervall. Fundera på vilken typ av prediktioner som är lämpligast.

In [ ]:
res.fit().predict(U_0)
#Konfidens eller prediktionsintervall, titta i hjälp-texten för vad obs och alpha gör.
res.fit().get_prediction(U_0).conf_int(obs=True/False, alpha=?)
#Allting på en gång
res.fit().get_prediction(U_0).summary_frame(alpha=?)

5. Residual variansen i regressionen, $\sigma^2$, fås ur *res.fit().scale*.